In [82]:
import pandas as pd
import numpy as np
import os
import sys
import argparse
import logging
import json
import re

In [83]:
# # Create an output folder if it doesn't exist
# output_folder = os.path.join(os.path.dirname(__file__), 'output')
# if not os.path.exists(output_folder):
#     os.makedirs(output_folder)

In [84]:
# Google Sheets API
from googleapiclient.discovery import build
from google.oauth2 import service_account
from google.auth.transport.requests import Request
from google.auth.exceptions import RefreshError

In [85]:
import os

# Assuming your credentials are in a folder named 'config' in the current working directory
config_folder = os.path.join(os.getcwd(), 'config')
credentials_path = os.path.join(config_folder, 'credentials.json')

if not os.path.exists(credentials_path):
    print(f"Credentials file not found: {credentials_path}")
else:
    print(f"Credentials path found: {credentials_path}")

Credentials path found: /Users/mananbhargava/Documents/Workspaces/dan-cs-ed/remind/gradesync_input/config/credentials.json


In [86]:
# Sheet & credentials config
google_sheet_id = '11H0hRtJOHCy59jaxbdRSp7JhDtkaiOibvexZ4Shj4wE'
google_sheet_credentials = 'credentials.json'
config_folder = os.path.join(os.getcwd(), 'config')
credentials_path = os.path.join(config_folder, 'credentials.json')

if not os.path.exists(credentials_path):
    logging.error(f"Credentials file not found: {credentials_path}")
    sys.exit(1)

In [87]:
def get_credentials():
    try:
        creds = service_account.Credentials.from_service_account_file(
            credentials_path,
            scopes=['https://www.googleapis.com/auth/spreadsheets.readonly']
        )
        return creds
    except Exception as e:
        logging.error(f"Error getting credentials: {e}")
        sys.exit(1)

def get_all_tab_names(sheet_id, credentials):
    try:
        service = build('sheets', 'v4', credentials=credentials)
        spreadsheet = service.spreadsheets().get(spreadsheetId=sheet_id).execute()
        sheet_metadata = spreadsheet.get('sheets', [])
        tab_names = [sheet['properties']['title'] for sheet in sheet_metadata]
        return tab_names
    except Exception as e:
        logging.error(f"Error fetching tab names: {e}")
        sys.exit(1)

def get_google_sheet_data(sheet_id, range_name, credentials):
    try:
        service = build('sheets', 'v4', credentials=credentials)
        sheet = service.spreadsheets()
        result = sheet.values().get(spreadsheetId=sheet_id, range=range_name).execute()
        return result.get('values', [])
    except Exception as e:
        logging.error(f"Error getting data from Google Sheet: {e}")
        sys.exit(1)

def convert_to_dataframe(data):
    """
    Convert raw sheet data to a cleaned DataFrame, padding short rows
    """
    try:
        headers = data[0]
        rows = data[1:]

        # Pad shorter rows with empty strings to match header length
        padded_rows = [row + [''] * (len(headers) - len(row)) for row in rows]

        df = pd.DataFrame(padded_rows, columns=headers)
        return df
    except Exception as e:
        logging.error(f"Error converting data to DataFrame: {e}")
        sys.exit(1)

In [88]:
creds = get_credentials()
creds

In [89]:
tab_names = get_all_tab_names(google_sheet_id, creds)
tab_names

['Roster',
 'Labs',
 'Pyturis',
 'Discussions',
 'Sheet8',
 'Projects',
 'Lecture Quizzes',
 'Midterms',
 'Postterms',
 'Lecture Quiz 4: Iteration',
 'Lecture Quiz 12: Higher-Order Functions',
 'Lab 4: Lists + Loops (Conceptual)',
 'Lab 4: Lists + Loops (code)',
 'Postterm 3: With Snap! (HOFs)',
 'Postterm 3: With Snap! (Recursion)',
 'Lab 5: Lists + HOFs (Conceptual)',
 'Lecture Quiz 21: Generative AI',
 'Project 3: 2048',
 'Lecture Quiz 2: Functions',
 'Lecture Quiz 1: Welcome to CS10 u0026 Abstraction',
 'Lab 5: Lists + HOFs (Code)',
 'Lab 2: Build Your Own Blocks (Conceptual)',
 'Lab 2: Build Your Own Blocks (Code)',
 'Discussion 1: Welcome to CS10!',
 'Lab 8: Boards (Conceptual)',
 'Discussion 2: Numeral Representation + Control Structurest',
 'Discussion 3: Domain/Range, Scoping, Iteration, Listst',
 'Project 1: Wordlexe2x84xa2-lite',
 'Lab 3: Conditionals, Reporters, u0026 Abstraction (Conceptual)',
 'Lab 3: Conditionals, Reporters u0026 Abstraction (Code)',
 'Lab 6: Algorithms 

In [90]:
tab = 'Project 3: 2048'
if tab not in tab_names:
    logging.error(f"Tab name '{tab}' not found in the Google Sheet.")
    sys.exit(1)

In [91]:
print(f"\n Parsing tab: {tab}")
range_str = f"{tab}"
raw_data = get_google_sheet_data(google_sheet_id, range_str, creds)
raw_data
# if not raw_data or len(raw_data) < 2:
#   print(f"Skipping {tab} (no data)")


 Parsing tab: Project 3: 2048


[["b'First Name",
  'Last Name',
  'SID',
  'Email',
  'Sections',
  'Total Score',
  'Max Points',
  'Status',
  'Submission ID',
  'Submission Time',
  'Lateness (H:M:S)',
  'View Count',
  'Submission Count',
  '1: Autograder (35.0 pts)'],
 ['Anita',
  'Qian',
  '3040005244',
  'anita_qian@berkeley.edu',
  'compsci-10-dis-801-in-person',
  '35',
  '35',
  'Graded',
  '314621931',
  '2025-03-05 10:45:24 -0800',
  '0:00:00',
  '0',
  '1',
  '35'],
 ['Malachite',
  'McEvoy',
  '3036547996',
  'malachitemcevoy@berkeley.edu',
  'compsci-10-dis-801-in-person',
  '35',
  '35',
  'Graded',
  '314680106',
  '2025-03-05 13:17:20 -0800',
  '0:00:00',
  '0',
  '2',
  '35'],
 ['Abigail',
  'Johnson',
  '3040928660',
  'abigail.johnson@berkeley.edu',
  'compsci-10-dis-801-in-person',
  '35',
  '35',
  'Graded',
  '314713498',
  '2025-03-05 14:46:33 -0800',
  '0:00:00',
  '0',
  '1',
  '35'],
 ['Jack',
  'Hulse',
  '3040081294',
  'johnhulse@berkeley.edu',
  'compsci-10-dis-801-in-person',
  '35',

In [92]:
df = convert_to_dataframe(raw_data)
df.head()

,b'First Name,Last Name,SID,Email,Sections,Total Score,Max Points,Status,Submission ID,Submission Time,Lateness (H:M:S),View Count,Submission Count,1: Autograder (35.0 pts)
0,Anita,Qian,3040005244,anita_qian@berkeley.edu,compsci-10-dis-801-in-person,35,35,Graded,314621931,2025-03-05 10:45:24 -0800,0:00:00,0,1,35
1,Malachite,McEvoy,3036547996,malachitemcevoy@berkeley.edu,compsci-10-dis-801-in-person,35,35,Graded,314680106,2025-03-05 13:17:20 -0800,0:00:00,0,2,35
2,Abigail,Johnson,3040928660,abigail.johnson@berkeley.edu,compsci-10-dis-801-in-person,35,35,Graded,314713498,2025-03-05 14:46:33 -0800,0:00:00,0,1,35
3,Jack,Hulse,3040081294,johnhulse@berkeley.edu,compsci-10-dis-801-in-person,35,35,Graded,314713498,2025-03-05 14:46:33 -0800,0:00:00,0,1,35
4,Kelly,Chou,3038568093,kelly0919@berkeley.edu,compsci-10-dis-801-in-person,30.35,35,Graded,314886910,2025-03-06 00:34:41 -0800,0:00:00,0,1,30.35


In [93]:
def clean_assignment_df(df, tab_name):
    """
    Cleans and formats a Google Sheets assignment DataFrame:
    - Filters relevant columns
    - Adds an 'assignment' column using the provided tab name
    - Renames columns, reorders them, and standardizes the formatting

    Args:
        df (pd.DataFrame): Raw DataFrame from Google Sheets
        tab_name (str): Name of the Google Sheet tab (used as assignment label)

    Returns:
        pd.DataFrame: Cleaned and formatted DataFrame
    """
    required_columns = ["b'First Name", 'Last Name', 'SID', 'Email', 'Status', 'Submission Time', 'Lateness (H:M:S)']
    missing = [col for col in required_columns if col not in df.columns]
    if missing:
        raise ValueError(f"Missing columns in input DataFrame: {missing}")
    
    # Filter and format
    filtered_df = df[required_columns].copy()
    filtered_df['Assignment'] = tab_name
    filtered_df = filtered_df[['Assignment'] + required_columns]
    filtered_df.rename(columns={"b'First Name": 'First Name'}, inplace=True)
    filtered_df.columns = filtered_df.columns.str.lower().str.strip()

    return filtered_df

In [94]:
cleaned_df = clean_assignment_df(df, tab)
cleaned_df

,assignment,first name,last name,sid,email,status,submission time,lateness (h:m:s)
0,Project 3: 2048,Anita,Qian,3040005244,anita_qian@berkeley.edu,Graded,2025-03-05 10:45:24 -0800,0:00:00
1,Project 3: 2048,Malachite,McEvoy,3036547996,malachitemcevoy@berkeley.edu,Graded,2025-03-05 13:17:20 -0800,0:00:00
2,Project 3: 2048,Abigail,Johnson,3040928660,abigail.johnson@berkeley.edu,Graded,2025-03-05 14:46:33 -0800,0:00:00
3,Project 3: 2048,Jack,Hulse,3040081294,johnhulse@berkeley.edu,Graded,2025-03-05 14:46:33 -0800,0:00:00
4,Project 3: 2048,Kelly,Chou,3038568093,kelly0919@berkeley.edu,Graded,2025-03-06 00:34:41 -0800,0:00:00
...,...,...,...,...,...,...,...,...
60,Project 3: 2048,Kaixing,Zhang,3040937812,keson@berkeley.edu,Missing,,
61,Project 3: 2048,Erin,ZHOU,3040947731,uerinzhou@berkeley.edu,Missing,,
62,Project 3: 2048,Efe,Atli,23772813,eatli@berkeley.edu,Missing,,
63,Project 3: 2048,Felipe,Ruiz-Tagle,3040705060,felipe.ruiztagle@berkeley.edu,Missing,,


In [95]:
# Get the output folder
import os

# Assuming your credentials are in a folder named 'output' in the current working directory
output_folder = os.path.join(os.getcwd(), 'output')
output_folder

'/Users/mananbhargava/Documents/Workspaces/dan-cs-ed/remind/gradesync_input/output'

In [ ]:
output_filename = f"{tab}.csv"
output_path = os.path.join(output_folder, output_filename)
cleaned_df.to_csv(output_path, index=False)
print(f"Saved {output_path}")
print(f"Saved {tab}.csv")

Saved /Users/mananbhargava/Documents/Workspaces/dan-cs-ed/remind/gradesync_input/output/Project 3: 2048.csv
Saved Project 3: 2048.csv
